In [11]:
import numpy as np

In [12]:
data = np.loadtxt("data/spambase.data", dtype='f', delimiter=',')

print (data)

max_value_in_data = [ 0 ] * 58

for lin in data:
    for x in range(0, 58):
        max_value_in_data[x] = max( max_value_in_data[x], lin[x] )
        

[[0.000e+00 6.400e-01 6.400e-01 ... 6.100e+01 2.780e+02 1.000e+00]
 [2.100e-01 2.800e-01 5.000e-01 ... 1.010e+02 1.028e+03 1.000e+00]
 [6.000e-02 0.000e+00 7.100e-01 ... 4.850e+02 2.259e+03 1.000e+00]
 ...
 [3.000e-01 0.000e+00 3.000e-01 ... 6.000e+00 1.180e+02 0.000e+00]
 [9.600e-01 0.000e+00 0.000e+00 ... 5.000e+00 7.800e+01 0.000e+00]
 [0.000e+00 0.000e+00 6.500e-01 ... 5.000e+00 4.000e+01 0.000e+00]]


Isso mostra que teremos que tratar os dados antes de rodar __Online SVM via Online Gradient Descent__

Temos ao menos duas etapas:
1 - Transformar os valores de classe ( variável target ) de 0, 1 para -1 ,1
2 - Temos que fazer uma normalização dos parâmetros. Pois alguns dados são percentuais e estão no range [0, 100] e outros podem atingir valores maiores como por exemplo 15841. Não queremos que um atributo tenha uma importância maior que outro por ser representado em uma escala diferente. Vamos fazer uma normalização __hard__ que é mapear o maior valor de uma determinada feature para 1 e dividir os outros valores dessa feature específica pelo maior valor encontrado.

Se a normalização hard se mostrar ruim na prática, podemos pensar em outras normalizações. ( Soft usando mediana e desvio-padrão )..



In [13]:
# vamos alterar os valores da variável target de (0, 1) para (-1, 1)
for lin in data:
    if lin[57] == 0:
        lin[57] = -1
        
# Agora vamos fazer a normalizacao hard
normalized_data = data

for lin in data:
    for x in range(0, 57):
        max_value_in_data[x] = max( max_value_in_data[x], lin[x] )
        
for lin in normalized_data:
    for x in range(0, 57):
        lin[x] = (lin[x] / max_value_in_data[x] )
        

Breve descrição de Online SVM via Online Gradient Descent
* No instante t
    * Selecionamos um vetor pt
    * Recebemos uma nova instância do dataset (yt, zt), onde yt são os atributos e zt é a classe correspondente
    * Vamos tomar uma Hinge Loss definida como max(0, 1 - zt * <pt, yt> )
    * Utilizaremos o gradiente da função de perda do instante t para calcular o próximo p(t+1)
    
Vamos também utilizar sempre pt pertencente ao espaço euclideano de norma <= 1.

A princípio o p0 pode ser definido arbitrariamente, mas como vimos descrito em alguns lugares da literatura a inicialização com o vetor nulo vamos adotar essa estratégia.
   

In [14]:
# Aqui algumas funções auxiliares úteis
dimension = 57

def dot_product( a, b ):
    ans = 0
    for x in range(dimension):
        ans += ( a[x] * b[x] )
    return ans

def hinge_loss( p_t, z_t, y_t):
    return max(0, 1 - dot_product(p_t, y_t) * z_t )

def euclidean_norm( p_t ):
    ans = 0
    for coord in p_t:
        ans += (coord ** 2)
    return sqrt( ans )

def project_into_euclidean_norm_1( p_t ):
    norm = euclidean_norm( p_t )
    if norm <= 1:
        return p_t
    
    norm_p = p_t
    for coord in norm_p:
        coord = ( coord / norm )
    return norm_p



In [ ]:
# Vamos armazenar os pt obtidos ao longo do tempo, para poder analisar quão rápido o método alcança bons classificadores
# E no final vamos testar os valores obtidos como classificadores para o dataset completo e analisar a acurácia
all_p = []

current_p = [0] * 57
all_p.append(current_p)

# Agora vamos processar o dataset uma instancia de cada vez, num setup online
for instance in normalized_data:
    instance_features = instance[0:57] # slices are semi-open intervals (:, [0, 57)
    target_class = instance[57]
    

